In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, accuracy_score, f1_score, mean_squared_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from scipy.sparse import csr_matrix
from sklearn.svm import SVR

Now we download the data that was prepared by using the PaDEL tool. This dataset consists of >800 PaDEL features that describe PubChem finger prints.

In [3]:
df = pd.read_csv('bioactivity_data_all3_pubchem.csv')

**The dataset has 881 features and 1 target variable (pIC50)**

In [4]:
X = df.drop('pIC50', axis=1)
X

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4690,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4691,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4692,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4693,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
Y = df.pIC50
Y

,pIC50
0,6.124939
1,7.000000
2,4.301030
3,6.522879
4,6.096910
...,...
4690,5.612610
4691,5.595166
4692,5.419075
4693,5.460924


**Filter the dataset using Variance threshold**

In [6]:
# For eg: If the variance threshold kept at 0, to remove features. This removes any features with 0 variance (all identical feature values)
selection = VarianceThreshold(threshold=(0.85 * (1-0.85)))
X = selection.fit_transform(X)

In [7]:
X.shape

(4695, 173)

# **Train-test split**

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [9]:
X_train.shape, Y_train.shape

((3756, 173), (3756,))

# **Fitting a Random forest model on the data**

In [30]:


param_grid = {
    'n_estimators': [50, 100, 200],         # Number of trees in the forest
    'max_depth': [10, 20, None],            # Maximum depth of the tree (None means unlimited)
    'min_samples_split': [2, 5],            # Minimum number of samples required to split an internal node
    'random_state': [150]                   # Keep seed constant
}


#Initialize the Model and GridSearchCV

rf_model = RandomForestRegressor()
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    verbose=2,
    n_jobs=1
)


print("Starting Grid Search...")

grid_search.fit(X_train, Y_train)
print("Grid Search Complete.")



print("\nBest Parameters found on training set:")
print(grid_search.best_params_)


print(f"Best Cross-Validated R-squared: {grid_search.best_score_:.4f}")


best_rf_model = grid_search.best_estimator_

# Predict and evaluate on the unseen Test Set
Y_pred_tuned = best_rf_model.predict(X_test)


r2_tuned = r2_score(Y_test, Y_pred_tuned)
print(f"\nFinal Test Set R-squared (Tuned RF): {r2_tuned:.4f}")


mse_tuned = mean_squared_error(Y_test, Y_pred_tuned)
print(f"Final Test Set Mean Squared Error (Tuned RF): {mse_tuned:.4f}")

Starting Grid Search...
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Grid Search Complete.

Best Parameters found on training set:
{'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 200, 'random_state': 150}
Best Cross-Validated R-squared: 0.5355

Final Test Set R-squared (Tuned RF): 0.5295
Final Test Set Mean Squared Error (Tuned RF): 1.1013


# **Fitting a SVM model on the data**

In [10]:


np.random.seed(150)

# SVR model
svr = SVR()


param_grid_svr = {
    'kernel': ['rbf', 'linear'],  # RBF usually works best for non-linear data
    'C': [1, 10, 50],               # Regularization parameter
    'epsilon': [0.1, 0.2, 0.5],    # Margin of tolerance
    'gamma': ['scale', 'auto']            # Kernel coefficient (used only for rbf/poly)
}

# GridSearchCV setup
grid_search_svr = GridSearchCV(
    estimator=svr,
    param_grid=param_grid_svr,
    scoring='r2',
    cv=3,
    verbose=2,
    n_jobs=1)

# Fit the grid search
grid_search_svr.fit(X_train, Y_train)


print("Best SVR parameters:", grid_search_svr.best_params_)
print("Best CV R2 score:", grid_search_svr.best_score_)


Y_pred_svr = grid_search_svr.predict(X_test)

r2_svr = r2_score(Y_test, Y_pred_svr)
mse_svr = mean_squared_error(Y_test, Y_pred_svr)
print(f"SVM R-squared on test set: {r2_svr}")
print(f"SVM Mean Squared Error on test set: {mse_svr}")


Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] END ..........C=1, epsilon=0.1, gamma=scale, kernel=rbf; total time=   1.4s
[CV] END ..........C=1, epsilon=0.1, gamma=scale, kernel=rbf; total time=   1.4s
[CV] END ..........C=1, epsilon=0.1, gamma=scale, kernel=rbf; total time=   1.4s
[CV] END .......C=1, epsilon=0.1, gamma=scale, kernel=linear; total time=   1.6s
[CV] END .......C=1, epsilon=0.1, gamma=scale, kernel=linear; total time=   2.6s
[CV] END .......C=1, epsilon=0.1, gamma=scale, kernel=linear; total time=   1.7s
[CV] END ...........C=1, epsilon=0.1, gamma=auto, kernel=rbf; total time=   1.5s
[CV] END ...........C=1, epsilon=0.1, gamma=auto, kernel=rbf; total time=   1.4s
[CV] END ...........C=1, epsilon=0.1, gamma=auto, kernel=rbf; total time=   1.4s
[CV] END ........C=1, epsilon=0.1, gamma=auto, kernel=linear; total time=   1.7s
[CV] END ........C=1, epsilon=0.1, gamma=auto, kernel=linear; total time=   1.7s
[CV] END ........C=1, epsilon=0.1, gamma=auto, 

Best SVR parameters: {'C': 10, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf'}

Best CV R2 score: 0.5212583959975471

SVM R-squared on test set: 0.5169252047717257
SVM Mean Squared Error on test set: 1.1192617175748445

# **Fitting a Linear regressor model on the data**

In [ ]:

linear_regressor = LinearRegression()
linear_regressor.fit(X_train, Y_train)

Y_pred_linear = linear_regressor.predict(X_test)

r2_linear = r2_score(Y_test, Y_pred_linear)
print(f"Linear Regression R-squared: {r2_linear}")

mse_linear = mean_squared_error(Y_test, Y_pred_linear)
print(f"Linear Regression Mean Squared Error: {mse_linear}")


Linear Regression R-squared: 0.3402653092582044
Linear Regression Mean Squared Error: 1.5606268035219517


# **Fitting a XGBoost model on the data**

In [22]:


np.random.seed(150)

param_grid_xgb = {
    'n_estimators': [100, 200, 300],        # Number of boosting stages (trees)
    'max_depth': [3, 5, 7],                 # Maximum depth of the tree
    'learning_rate': [0.01, 0.1, 0.2],      # Step size shrinkage
    'random_state': [150]
}

xgb_model = xgb.XGBRegressor(
    n_jobs=1,
    objective='reg:squarederror' )

grid_search_xgb = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='r2',
    cv=5,
    verbose=2,
    n_jobs=1
)


print("Starting XGBoost Grid Search (5-fold CV)...")

grid_search_xgb.fit(X_train, Y_train)

print("Grid Search Complete.")
print("\nBest XGBoost Parameters found on training set:")

print(grid_search_xgb.best_params_)

print(f"Best Cross-Validated R-squared: {grid_search_xgb.best_score_:.4f}")

best_xgb_model = grid_search_xgb.best_estimator_

# Predict and evaluate on the unseen Test Set
Y_pred_tuned_xgb = best_xgb_model.predict(X_test)


r2_tuned_xgb = r2_score(Y_test, Y_pred_tuned_xgb)
print(f"\nFinal Test Set R-squared (Tuned XGBoost): {r2_tuned_xgb:.4f}")


mse_tuned_xgb = mean_squared_error(Y_test, Y_pred_tuned_xgb)
print(f"Final Test Set Mean Squared Error (Tuned XGBoost): {mse_tuned_xgb:.4f}")

Starting XGBoost Grid Search (5-fold CV)...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, random_state=150; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, random_state=150; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, random_state=150; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, random_state=150; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, random_state=150; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=200, random_state=150; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=200, random_state=150; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=200, random_state=150; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=200, random_state=150; total time=   0.2s
[CV] END learning_rat

Best XGBoost Parameters found on training set:

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'random_state': 150}

Best Cross-Validated R-squared: 0.5281

Final Test Set R-squared (Tuned XGBoost): 0.5386
Final Test Set Mean Squared Error (Tuned XGBoost): 1.0914

**INFERENCE:**



On the whole,  The best models could only explain > 50% of the variability in the pIC50 values. But, this anyways proves that that a Structure-Activity Relationship (SAR) **does exist** between the fingerprints and the bioactivity.

This is likely due to experimental noise in the original ChEMBL IC50 measurements, which the ML model cannot filter out. Bioactivity data from public databases like ChEMBL is heterogeneous. Different labs use different assay protocols, cell lines, and IC50 calculation methods. This is the probable source of inherent noise in these experimental values.